In [1]:
from __future__ import print_function

In [2]:
def to_bin(x):
    vv = [ (x>>(32-(i+1)*4))&0xF for i in range(8)]
    return '{0:04b}_{1:04b}|{2:04b}_{3:04b}|{4:04b}_{5:04b}|{6:04b}_{7:04b}'.format( *vv )

In [3]:
import uio
reload(uio)
import numpy as np

axi_dma_direct_dtype = np.dtype([
        ('cr'        , '<u4'),
        ('st'        , '<u4'),
        ('reserved1' , '<u4',4),
        ('addr_lo'   , '<u4'),
        ('addr_hi'   , '<u4'),
        ('reserved2' , '<u4',2),
        ('length'    , '<u4'),
        ('pad'       , '<u4'),])

def dma_reset(dma):
    dma.cr = 4
    dma.cr = 0
    
def dma_idle(dma):
    return (dma.st&2) == 2

def dma_halted(dma):
    return (dma.st&1) == 1

In [4]:
mem_uio = uio.UIO('scratch_mem')

dma_uio = uio.UIO('dma')
mm2s, s2mm = dma_uio.as_recarray(2, dtype=axi_dma_direct_dtype)

#reset dma streams
dma_reset(mm2s)
dma_reset(s2mm)

print('\n'.join([to_bin(x) for x in [mm2s.cr, s2mm.cr, mm2s.st,s2mm.st]]))

dma_uio, mem_uio, mm2s, s2mm, dma_idle(s2mm)

0000_0000|0000_0001|0000_0000|0000_0010
0000_0000|0000_0001|0000_0000|0000_0010
0000_0000|0000_0000|0000_0000|0000_0001
0000_0000|0000_0000|0000_0000|0000_0001


(UIO: dma (uio2) sz:0x10000 @0x40400000,
 UIO: scratch_mem (uio0) sz:0x2000000 @0x3e000000,
 (65538L, 1L, array([0, 0, 0, 0], dtype=uint32), 0L, 0L, array([0, 0], dtype=uint32), 0L, 0L),
 (65538L, 1L, array([0, 0, 0, 0], dtype=uint32), 0L, 0L, array([0, 0], dtype=uint32), 0L, 0L),
 False)

In [16]:
SZ=(1<<19)

data = mem_uio.as_ndarray()

src_data = data[:SZ]
dst_data = data[SZ:SZ*2]

src_data[:] = np.random.randint(0,256,SZ)
dst_data[:] = 0xFF

[hex(x) for x in [mem_uio.phy(), mem_uio.phy(src_data), mem_uio.phy(dst_data)]]

print('\n'.join([to_bin(x) for x in [mm2s.cr, s2mm.cr, mm2s.st,s2mm.st]]))
print("SRC:",src_data[:4])
print("DST:",dst_data[:4])

0000_0000|0000_0001|0000_0000|0000_0010
0000_0000|0000_0001|0000_0000|0000_0010
0000_0000|0000_0000|0000_0000|0000_0001
0000_0000|0000_0000|0000_0000|0000_0001
SRC: [197 175 200  37]
DST: [255 255 255 255]


In [21]:
map(dma_reset, [s2mm,mm2s])
map(lambda dma: print(to_bin(dma.st)), [s2mm,mm2s])

#set up stream to memory transfer first
s2mm.addr_lo = mem_uio.phy(dst_data)
s2mm.cr = (0x1<<12)|1 #on completion interrupt
s2mm.length = SZ

assert dma_idle(s2mm) ==  False

#now do memory to stream
mm2s.addr_lo = mem_uio.phy(src_data)
mm2s.cr = (0x1<<12)|1 #on completion interrupt
mm2s.length = SZ

x = dma_uio.wait_for_interrupt()
print('int:',x)
if x < 2: #wait for second interrupt
    x = dma_uio.wait_for_interrupt()
    print('int:',x)


#poll until 'idle' bit is flipped on the PL->PS dma channel
while dma_idle(s2mm) == False:
    print('.', end='')
print(' done')
assert dma_idle(s2mm)

map(lambda dma: print(to_bin(dma.st)), [s2mm,mm2s])

print(' resetting')
dma_reset(s2mm)
dma_reset(mm2s)

map(lambda dma: print(to_bin(dma.st)), [s2mm,mm2s])
pass

0000_0000|0000_0000|0000_0000|0000_0001
0000_0000|0000_0000|0000_0000|0000_0001
int: 1
int: 1
 done
0000_0000|0000_0000|0001_0000|0000_0010
0000_0000|0000_0000|0001_0000|0000_0010
 resetting
0000_0000|0000_0000|0000_0000|0000_0001
0000_0000|0000_0000|0000_0000|0000_0001


In [18]:
if (src_data == dst_data).all():
    print('SUCCESS: Data copied as expected')
else:
    print('FAILED: dst and src do not match up')

print("SRC:",src_data[:4])
print("DST:",dst_data[:4])

SUCCESS: Data copied as expected
SRC: [197 175 200  37]
DST: [197 175 200  37]


In [10]:
dma_uio.wait_for_interrupt()

KeyboardInterrupt: 

In [ ]:
%%sh

date
whoami
ls -l /dev/uio* /dev/mem
cat /sys/class/uio/uio?/maps/map0/name
cat /sys/class/uio/uio?/name
ls /sys/class/uio/uio0/maps/map0

In [ ]:
#!dtc -I fs -O dts /proc/device-tree/ 